### ToolNode란?
**ToolNode**는 LangGraph에서 **외부 도구들을 실행하는 특별한 노드**입니다.
- `자동화된 도구 실행`:  
    LLM이 생성한 도구 호출 요청(tool calls)을 감지하고, 해당 도구를 실제로 실행합니다.

- `State 업데이트`: 
    도구 실행 결과를 ToolMessage 형태로 만들어 그래프의 상태(State)에 추가합니다. 이를 통해 에이전트는 도구 사용 결과를 인지하고 다음 단계를 계획할 수 있습니다.

- `동시 실행 및 오류 처리`: 
    여러 도구를 동시에 호출하고 실행할 수 있으며, 도구 실행 중 발생하는 오류를 처리하는 기능도 내장되어 있습니다.

#### 동작방식
1. `State 입력받기`  
    - State를 전달 받고 
    - messages리스트의 가장 마지막 메시지를 찾음
2. `도구호출 확인`  
    - 해당 메시지가 AI 에이전트가 생성한 **AIMesasage** 이고, 
    - 안에 tool_calls 속성이 포함되어 있는지를 확인
3. `도구 실행`  
    - tool_calls가 있다면 해당하는 이름과 일치하는 도구를 찾고 실행
4. `State 업데이트`
    - 실행된 도구의 결과를 받아 **ToolMessage** 라는 형태로 응답
    - 실행한 도구이름, 실행결과, tool_call_id

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI API 환경변수 값 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"OPENAI_API_KEY가 설정되어 있나요?: {openai_api_key[:10]}...")

**Tavily 검색 도구 설정**

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Tavily 검색 도구 생성
tavily_tool = TavilySearchResults(
    max_results=3,  # 최대 검색 결과 수
    search_depth="advanced",  # 검색 깊이 설정
    include_answer=True,  # 직접적인 답변 포함
    include_raw_content=False,  # 원본 콘텐츠 제외 (크기 줄이기)
    include_images=False  # 이미지 제외
)

# 도구 목록 생성
tools = [tavily_tool]

**LLM 모델 설정 및 도구 바인딩**

In [ ]:
from langchain.chat_models import init_chat_model

# LLM 설정
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

# 도구바인딩
llm_with_tools = llm.bind_tools(tools)

**Chatbot Node 생성**

In [ ]:
# ChatBot Node 함수 정의
def chatbot_node(state: State):
    
    # 1. 현재 상태에서 모든 메시지 가져오기
    messages = state["messages"]
    
    # 2. LLM 호출하여 응답 생성
    response = llm_with_tools.invoke(messages)
    
    # 3. 생성된 응답을 새로운 상태로 반환
    return {"messages": [response]}

**상태(State) 정의**
- 검색 과정에서 필요한 정보를 추적하기 위한 상태를 정의합니다.

In [ ]:
from typing import Annotated, TypedDict
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class State(TypedDict):
    # 모든 대화 메시지들 (HumanMessage, AIMessage, ToolMessage 등)
    messages: Annotated[list[AnyMessage], add_messages]

**Tool Node**

In [ ]:
from langgraph.prebuilt import ToolNode


# ToolNode 초기화
tool_node = ToolNode(tools=tools)

**그래프 생성**

In [ ]:
from langgraph.graph import StateGraph, START, END

# 상태 그래프 생성
graph_builder = StateGraph(State)

# 챗봇 노드 추가
graph_builder.add_node("chatbot_node", chatbot_node)

# 도구 노드 추가
graph_builder.add_node("tools", tool_node)

#### 엣지 추가

# 시작 엣지 추가
graph_builder.add_edge(START, "chatbot_node")

# 도구 노드에서 챗봇 노드로 엣지 추가
graph_builder.add_edge("tools", "chatbot_node")

# 챗봇 노드에서 종료 엣지 추가
graph_builder.add_edge("chatbot_node", END)


### 조건부 분기 처리
**add_conditional_edges**
- 그래프 실행 중 특정 조건에 따라 다음에 실행할 노드를 다르게 연결할 수 있도록 할 수 있다.

**tools_condition의 동작 원리**
1. 상태(State) 입력받기:  
    - 그래프의 현재 상태(State) 객체를 입력으로 받습니다. 이 상태에는 대화 기록(messages)이 포함되어 있습니다.

2. 마지막 메시지 확인:  
    - 대화 기록(messages) 리스트에서 가장 마지막에 추가된 메시지를 확인합니다.

3. tool_calls 속성 검사: 
    - 만약 마지막 메시지에 tool_calls 속성이 있고, 내용이 비어있지 않다면, tools_condition은 미리 정해진 문자열(일반적으로 도구 실행 노드의 이름, 예: "tools")을 반환합니다.
    - 만약 tool_calls 속성이 없거나 비어있다면, END를 의미하는 문자열을 반환합니다.

In [ ]:
from langgraph.prebuilt import tools_condition

# 조건부 엣지 추가
graph_builder.add_conditional_edges(
    "chatbot_node",
    tools_condition # chatbot_node에서 "tools"로 갈지, "END"로 갈지 결정
)

# 그래프 컴파일
graph = graph_builder.compile()

**그래프 시각화**

In [ ]:
# 그래프 시각화
from IPython.display import Image, display

try:
    # 그래프 구조를 Mermaid 다이어그램으로 표시
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    pass

In [ ]:
user_input = "HelloworldLab 회사에 대해 알려주세요"
# 그래프 이벤트 스트리밍
for event in graph.stream({"messages": [("user", user_input)]}, stream_mode="values"):
    # 이벤트 값 출력
    for key, value in event.items():
        print(f"\n=================\nSTEP: {key}\n=================\n")
        print(value[-1])